In [7]:
using Laplacians
using DataStructures
include("../src/cutPageRank.jl")
include("../src/flowClustering.jl")
include("../src/cutHeuristics.jl")

dumb (generic function with 1 method)

In [9]:
function condTest(minSize)
    a = chimera(2500)
    s = prn(a, [1,2,3], 0.2, 5);
    conds = compConductance(a, s)
    if length(s) < minSize
        return -1,0,0,0
    end
    
    minEpsSigma = getVolume(a, s) / getVolume(a, setdiff(collect(1:max(a.n, a.m)), s))
    cut, flow = localImprove(a, s, epsSigma = minEpsSigma)
    condcut = compConductance(a, cut)
    
    heur_refcut = refineCut(a, s)
    condref = compConductance(a, heur_refcut)
    
    heur_dumb = dumb(a, s)
    conddumb = compConductance(a, heur_dumb)
    
    println(conds, " with ", length(s), " vertices")
    println(condcut, " with ", length(cut), " vertices. improvement = ", (conds - condcut) / conds * 100, " <- flow")
    println(condref, " with ", length(heur_refcut), " vertices. improvement = ", (conds - condref) / conds * 100)
    println(conddumb, " with ", length(heur_dumb), " vertices. improvement = ", (conds - conddumb) / conds * 100)
    println("*********")
    
    
    return conds, condcut, condref, conddumb
end

initial = []
flowbased = []
heurbased = []
dumbbased = []

for i in 1:300
    println("Test ", i)
    
    x,y,z,t = condTest(30)
    if x == -1
        continue
    end
    
    push!(initial, x)
    push!(flowbased, y)
    push!(heurbased, z)
    push!(dumbbased, t)
end

Test 1
0.1998320738874895 with 794 vertices
0.15160796324655437 with 653 vertices. improvement = 24.132317551829303 <- flow
0.15638963360142985 with 746 vertices. improvement = 21.739473269200438
0.16358253145817914 with 1351 vertices. improvement = 18.140002114835564
*********
Test 2
Test 3
Test 4
0.19941775836972345 with 133 vertices
0.1263537906137184 with 112 vertices. improvement = 36.63864660465362 <- flow
0.15328467153284672 with 134 vertices. improvement = 23.133890990462998
0.14806201550387596 with 254 vertices. improvement = 25.75284332031914
*********
Test 5
Test 6
0.1975806451612903 with 90 vertices
0.11467116357504216 with 105 vertices. improvement = 41.96234986406029 <- flow
0.14929577464788732 with 95 vertices. improvement = 24.438056912906006
0.15178571428571427 with 180 vertices. improvement = 23.17784256559767
*********
Test 7
Test 8
Test 9
Test 10
Test 11
0.19755244755244755 with 143 vertices
0.12438423645320197 with 203 vertices. improvement = 37.037359954662364 <- 

In [10]:
println(mean(initial), " ", median(initial))
println(mean(flowbased), " ", median(flowbased))
println(mean(heurbased), " ", median(heurbased))
println(mean(dumbbased), " ", median(dumbbased))


0.19455026070986756 0.19755244755244755
0.05808431699868943 0.02830540037243948
0.15159973840123817 0.1592356687898089
0.14067997183413733 0.14806201550387596


In [ ]:
a = chimera(500);
@time x, y = spectralCoords(a);

In [ ]:
include("../src/cutPageRank.jl")
@time s = prn(a, [100,101,102], 0.2, 4);
plotGraph(a,x,y,[0,0,1])
plotGraph(a[s,s],x[s],y[s],[1,0,0],setaxis=false)

In [ ]:
include("../src/flowClustering.jl")
epsSigma = 10.0

@time cut, flow = localImprove(a, s, epsSigma)

println(compConductance(a, s))
println(compConductance(a, cut))

plotGraph(a,x,y,[0,0,1])
plotGraph(a[cut,cut],x[cut],y[cut],[1,0,0],setaxis=false)

In [ ]:
function condTest()
    a = chimera(100);
    s = prn(a, [1,2,3], 0.2, 5);
    if length(s) == 0
        return
    end
    
    # construct the vector of test EpsSigma values
    minEpsSigma = getVolume(a, s) / getVolume(a, setdiff(collect(1:max(a.n, a.m)), s))
    aux = [minEpsSigma + 0.0001, minEpsSigma + 0.02, 0.05, 0.1, 0.2, 0.4, 1.0, 2.0, 4.0, 10.0, 30.0]
    epsSigma = []
    for eS in aux
        if eS > minEpsSigma
            push!(epsSigma, eS)
        end
    end
    sort!(epsSigma)
    
    refcut = refineCut(a, s)
    condref = compConductance(a, refcut)
    
    # do the testing
    for eS in epsSigma
        cut, flow = localImprove(a, s, epsSigma = eS)
       
        conds = compConductance(a, s)
        condcut = compConductance(a, cut)
        
        if eS == minEpsSigma + 0.0001
#             println(conds, " ", condcut, " improvement(epsig = ", eS,") = ", 100 * (conds - condcut) / conds, "%", " initial set size = ", length(s), " improved set size = ", length(cut))
#             println(conds, " ", condref, " improvement = ", 100 * (conds - condref) / conds, "%", " initial set size = ", length(s), " improved set size = ", length(refcut))
#             println("!!!!")
        end
        if (conds < condcut)
            for iter in 1:100
                println("@@@@@@@@@@@@@@@@@@@@")
            end
            println(conds, " ", condcut, " improvement(epsig = ", eS,") = ", 100 * (conds - condcut) / conds, "%", " initial set size = ", length(s), " improved set size = ", length(cut))
            println(a)
            println(s)
        end
    end
end

for i in 1:10000
    if i % 100 == 1
        println("Test ", i)
    end
#     println("Test ", i)
    condTest()
#     println("*************************")
end